<h1><b>Q-Learning</b></h1>
<p align="justify">Στην συγκεκριμένη άσκηση θα μελετήσετε το στοχαστικό αλγόριθμο <i><a href="https://en.wikipedia.org/wiki/Q-learning">Q-Learning</a></i>, χρησιμοποιώντας το έτοιμο πρόγραμμα που σας δίνεται. Με το συγκεκριμένο πρόγραμμα μπορείτε να βρείτε τη μικρότερη διαδρομή μεταξύ δύο σημείων όπως φαίνεται στο παρακάτω σχήμα (στο δοθέν παράδειγμα η αρχή είναι το σημείο 0 και ο τελικός προορισμός το σημείο 7):</p>
<img src="https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab6/ex6a.PNG"></img>
<p align="justify">Με βάση το τον κώδικα που σας έχει δοθεί, καλείστε να απαντήσετε στα παρακάτω ερωτήματα:</p>
<ul>
<li>Να περιγράψετε σύντομα τον αλγόριθμο <i>Q-Learning</i>. Σε ποια προβλήματα
θεωρείτε ότι ταιριάζει ως τρόπος εκμάθησης η <i><a href="https://en.wikipedia.org/wiki/Reinforcement_learning">Ενισχυτική Μάθηση (Reinforcement Learning)</a></i>; Ποια είναι η βασική διαφορά του αλγορίθμου <i>Q-Learning</i> από τους αλγορίθμους <i>Policy Iteration</i> και <i>Value Iteration</i>;</li>
<li>Σχεδιάστε τη <i>μήτρα διασύνδεσης των κόμβων</i> (με -1 θα συμβολίζετε τις περιπτώσεις όπου δεν υπάρχει διασύνδεση μεταξύ των κόμβων, με 0 την περίπτωση όπου υπάρχει απλή διασύνδεση μεταξύ των κόμβων και με 100 όταν ένας κόμβος διασυνδέεται με τον τερματικό κόμβο).</li>
<li>Εκτελέστε το πρόγραμμα που σας έχει δοθεί και στη συνέχεια αλλάξτε τον τρόπο που συνδέονται οι κόμβοι. Τι παρατηρείτε ως προς τα αποτελέσματα;</li>
<li>Έστω ότι οι κόμβοι έχουν και πρόσθετη πληροφορία (εκτός από τον τρόπο διασύνδεσης τους) που ο agent θα μπορεί να την αξιοποιήσει για να φτάσει στον προορισμό του. Τι θα συνέβαινε ως προς τα αποτελέσματα;</li>
</ul>

In [0]:
import numpy as np
import pylab as plt

# map cell to cell, add circular cell to goal point
points_list = [(0,1), (1,5), (5,6), (5,4), (1,2), (2,3), (2,7)]

# set the goal
goal = 7

# how many points in graph? x points
MATRIX_SIZE = 8

# create matrix x*y
R = np.matrix(np.ones(shape=(MATRIX_SIZE, MATRIX_SIZE)))
R *= -1

# assign zeros to paths and 100 to goal-reaching point
for point in points_list:
    print(point)
    if point[1] == goal:
        R[point] = 100
    else:
        R[point] = 0

    if point[0] == goal:
        R[point[::-1]] = 100
    else:
        # reverse of point
        R[point[::-1]]= 0

# add goal point round trip
R[goal,goal]= 100

Q = np.matrix(np.zeros([MATRIX_SIZE,MATRIX_SIZE]))

# learning parameter
gamma = 0.8

initial_state = 1

def available_actions(state):
    current_state_row = R[state,]
    av_act = np.where(current_state_row >= 0)[1]
    return av_act

available_act = available_actions(initial_state) 

def sample_next_action(available_actions_range):
    next_action = int(np.random.choice(available_act,1))
    return next_action

action = sample_next_action(available_act)

def update(current_state, action, gamma):
    
  max_index = np.where(Q[action,] == np.max(Q[action,]))[1]
  
  if max_index.shape[0] > 1:
      max_index = int(np.random.choice(max_index, size = 1))
  else:
      max_index = int(max_index)
  max_value = Q[action, max_index]
  
  Q[current_state, action] = R[current_state, action] + gamma * max_value
  print('max_value', R[current_state, action] + gamma * max_value)
  
  if (np.max(Q) > 0):
    return(np.sum(Q/np.max(Q)*100))
  else:
    return (0)
    
update(initial_state, action, gamma)

# Training
scores = []
for i in range(700):
    current_state = np.random.randint(0, int(Q.shape[0]))
    available_act = available_actions(current_state)
    action = sample_next_action(available_act)
    score = update(current_state,action,gamma)
    scores.append(score)
    print ('Score:', str(score))
    
print("Trained Q matrix:")
print(Q/np.max(Q)*100)

# Testing
current_state = 0
steps = [current_state]

while current_state != 7:

    next_step_index = np.where(Q[current_state,] == np.max(Q[current_state,]))[1]
    
    if next_step_index.shape[0] > 1:
        next_step_index = int(np.random.choice(next_step_index, size = 1))
    else:
        next_step_index = int(next_step_index)
    
    steps.append(next_step_index)
    current_state = next_step_index

print("Most efficient path:")
print(steps)

plt.plot(scores)
plt.show()